# Using the Edge Impulse Python SDK with TensorFlow and Keras

<!--- Do not modify the markdown for this example directly! It is generated from a notebook in https://github.com/edgeimpulse/notebooks --->

In [3]:
# If you have not done so already, install the following dependencies
!python -m pip install tensorflow==2.12.0 edgeimpulse

In [4]:
from tensorflow import keras
import edgeimpulse as ei
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842"
labels = ["nofire", "fire"]
num_classes = len(labels)
deploy_filename = "bushfire_detection_model_cpp.zip"

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Define paths for your dataset
train_val_directory = 'drive/MyDrive/forest_fire/Training and Validation'
test_directory = 'drive/MyDrive/forest_fire/Testing'

# Create an ImageDataGenerator for training and validation
train_val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalize pixel values
    validation_split=0.2      # Use 20% of the training data for validation
)

# Create generators for training and validation data
train_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),   # Resize images to 128x128
    batch_size=32,             # Number of images to return in each batch
    class_mode='binary',       # Binary classification (fire or no fire)
    subset='training'          # Set as training data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

validation_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    subset='validation'        # Set as validation data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 754 images belonging to 2 classes.
Found 187 images belonging to 2 classes.


In [10]:
# Create an ImageDataGenerator for testing (no split needed)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create a generator for the test data
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    shuffle=False              # Don't shuffle test data
)

Found 0 images belonging to 2 classes.


In [11]:
# Build your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Adjust epochs as needed
)

Epoch 1/10
23/23 [==============================] - 24s 973ms/step - loss: 0.5212 - accuracy: 0.9432 - val_loss: 0.0965 - val_accuracy: 0.9688
Epoch 2/10
23/23 [==============================] - 17s 757ms/step - loss: 0.0693 - accuracy: 0.9806 - val_loss: 0.0739 - val_accuracy: 0.9812
Epoch 3/10
23/23 [==============================] - 18s 756ms/step - loss: 0.0393 - accuracy: 0.9861 - val_loss: 0.0708 - val_accuracy: 0.9812
Epoch 4/10
23/23 [==============================] - 17s 731ms/step - loss: 0.0264 - accuracy: 0.9903 - val_loss: 0.0854 - val_accuracy: 0.9750
Epoch 5/10
23/23 [==============================] - 18s 775ms/step - loss: 0.0212 - accuracy: 0.9917 - val_loss: 0.0544 - val_accuracy: 0.9812
Epoch 6/10
23/23 [==============================] - 17s 729ms/step - loss: 0.0157 - accuracy: 0.9972 - val_loss: 0.0572 - val_accuracy: 0.9750
Epoch 7/10
23/23 [==============================] - 18s 775ms/step - loss: 0.0101 - accuracy: 0.9958 - val_loss: 0.0653 - val_accuracy: 0.9750

In [13]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy:.2f}')

ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
# List the available profile target devices
# ei.model.list_profile_devices()

In [ ]:
# Estimate the RAM, ROM, and inference time for our model on the target hardware family
try:
    profile = ei.model.profile(model=model,
                               device='openmv-h7p')
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

In [ ]:
# Define the output and input types for your model
labels = ["nofire", "fire"]  # Update the labels to match your classification task

model_output_type = ei.model.output_type.Classification(labels=labels)
model_input_type = ei.model.input_type.ImageData()  # Assuming images are the input type

# Define the filename for the deployment package
deploy_filename = "bushfire_detection_model.zip"

# Try to deploy the model directly as a C++ library in a .zip file
deploy_bytes = None
try:
    deploy_bytes = ei.model.deploy(
        model=model,  # The trained model
        model_output_type=model_output_type,
        model_input_type=model_input_type,
        deploy_target='zip'  # Export as a .zip file with C++ library
    )
    print("Model deployment successful.")
except Exception as e:
    print(f"Could not deploy: {e}")

# Write the deployed model bytes to a file
if deploy_bytes:
    with open(deploy_filename, 'wb') as f:
        f.write(deploy_bytes.getvalue())
    print(f"Model exported as {deploy_filename}")
else:
    print("Deployment failed; no deployable bytes were generated.")